In [ ]:
import h5py
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

## 1. Load Tensor

In [ ]:
file_path = "../data/test.h5"
with h5py.File(file_path, 'r') as file:
    data_halflife = file['halflife'][...]
    data_label = file['label'][...]
        
    tensor_halflife_test = torch.tensor(data_halflife)
    tensor_label_test = torch.tensor(data_label)

In [ ]:
load_path = '../tensor/tensor_promoter_test_embedding.pth'
tensor_promoter_test = torch.load(load_path)

## 2. Define model

In [ ]:
class CNNBinaryClassifier(nn.Module):
    def __init__(self):
        super(CNNBinaryClassifier, self).__init__()
        
        self.input_size = 5000
        
        self.conv1 = nn.Conv1d(1, 4, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(4, 16, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(16, 8, kernel_size=3, padding=1)  
        self.conv4 = nn.Conv1d(8, 4, kernel_size=3, padding=1)
        
        self.pool = nn.MaxPool1d(2, 2)
        
        self.dropout = nn.Dropout(0.5) 
        
        if self.input_size is not None:
            self.fc1 = nn.Linear(self.input_size, 256)
        else:
            self.fc1 = None
        self.fc2 = nn.Linear(256, 64)
        self.fc3 = nn.Linear(64, 1) 
        
        self.fc_y = nn.Linear(256,128)
        self.linear_y = nn.Linear(128,1)
    
    def forward(self, x, y):
        y = nn.functional.relu(self.fc_y(y)) 
        y = self.linear_y(y) 
        
        x = x.unsqueeze(2)

        concat_tensor = torch.concat((x,y),dim=1)
        concat_tensor = concat_tensor.unsqueeze(1)
        concat_tensor = concat_tensor.squeeze(-1)
        
        concat_tensor = self.pool(nn.functional.relu(self.conv1(concat_tensor)))
        concat_tensor = self.pool(nn.functional.relu(self.conv2(concat_tensor)))
        concat_tensor = self.pool(nn.functional.relu(self.conv3(concat_tensor)))  
        concat_tensor = self.pool(nn.functional.relu(self.conv4(concat_tensor)))  
        
        concat_tensor = concat_tensor.view(concat_tensor.size(0), -1)# 16215，5000
        
        if self.fc1 is None:  
            self.input_size = concat_tensor.size(1)  
            self.fc1 = nn.Linear(self.input_size, 256).to(concat_tensor.device)  
        
        concat_tensor = nn.functional.relu(self.fc1(concat_tensor))
        concat_tensor = self.dropout(concat_tensor) 
        concat_tensor = nn.functional.relu(self.fc2(concat_tensor))
        logit = self.fc3(concat_tensor) 
        
        return logit

## 3. Obtain results

In [ ]:
model = CNNBinaryClassifier().to('cpu')

model.load_state_dict(torch.load('../model/CNNpred.pth'))

model.eval()

tensor_halflife_test = tensor_halflife_test.float().to('cpu')
tensor_promoter_test = tensor_promoter_test.float().to('cpu')


with torch.no_grad():
    outputs = model(tensor_halflife_test,tensor_promoter_test)

probabilities = torch.sigmoid(outputs)

valid_labels = tensor_label_test.float()

predicted_labels = (probabilities > 0.55).float() 

# 计算各项指标
accuracy = accuracy_score(valid_labels.numpy(), predicted_labels.numpy())
precision = precision_score(valid_labels.numpy(), predicted_labels.numpy())
recall = recall_score(valid_labels.numpy(), predicted_labels.numpy())
f1 = f1_score(valid_labels.numpy(), predicted_labels.numpy())

auc = roc_auc_score(valid_labels.numpy(), probabilities.numpy())

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print(f'AUC: {auc:.4f}')